## Best Practices for Data Preprocessing

#### Always Explore & Visualize Data First

In [ ]:
from sklearn.datasets import fetch_california_housing
import pandas as pd
import numpy as np
data = fetch_california_housing(as_frame=True)
df = data.frame
df.iloc[0:100, 0] = np.nan  
X = df.drop(columns='MedHouseVal')
y = df['MedHouseVal']
missing_values = X.isnull().sum()
print("Missing values per column:\n", missing_values)
print("\nSummary statistics (to detect outliers):\n", X.describe())
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy='mean')),
    ('scaler', StandardScaler())
])

## Handle Missing & Inconsistent Data Before Applying ML Models

In [ ]:
from sklearn.datasets import fetch_california_housing
import pandas as pd
import numpy as np
data = fetch_california_housing(as_frame=True)
df = data.frame
df.iloc[0:100, 0] = np.nan 
X = df.drop(columns='MedHouseVal')
y = df['MedHouseVal']
X_dropped = X.dropna()
y_dropped = y[X_dropped.index] 
print("Shape after dropping missing values:", X_dropped.shape) 
from sklearn.impute import SimpleImputer
imputer = SimpleImputer(strategy='mean')
X_filled = imputer.fit_transform(X)
print("Shape after imputation:", X_filled.shape)

def cap_outliers(df, lower_percentile=0.01, upper_percentile=0.99):
    capped_df = df.copy()
    for col in df.columns:
        lower = df[col].quantile(lower_percentile)
        upper = df[col].quantile(upper_percentile)
        capped_df[col] = np.clip(df[col], lower, upper)
    return capped_df
X_capped = cap_outliers(pd.DataFrame(X_filled, columns=X.columns))
print("Outliers handled via capping.")

## Choose the Right Scaling Method

In [ ]:
from sklearn.preprocessing import MinMaxScaler
import pandas as pd
data = {'feature1': [10, 20, 30, 40, 50],
        'feature2': [1, 2, 3, 4, 5]}
df = pd.DataFrame(data)
scaler = MinMaxScaler()
scaled_data = scaler.fit_transform(df)
scaled_df = pd.DataFrame(scaled_data, columns=df.columns)
print(scaled_df)

from sklearn.preprocessing import RobustScaler
data = {'feature1': [10, 20, 30, 40, 500],  
        'feature2': [1, 2, 3, 4, 5]}

df = pd.DataFrame(data)
scaler = RobustScaler()
scaled_data = scaler.fit_transform(df)
scaled_df = pd.DataFrame(scaled_data, columns=df.columns)

print(scaled_df)
from sklearn.preprocessing import MaxAbsScaler
data = {'feature1': [-30, -20, 0, 20, 30],
        'feature2': [1, 2, 3, 4, 5]}
df = pd.DataFrame(data)
scaler = MaxAbsScaler()
scaled_data = scaler.fit_transform(df)
scaled_df = pd.DataFrame(scaled_data, columns=df.columns)
print(scaled_df)

## Keep Track of Data Transformations for Reproducibility

In [ ]:
import logging
from sklearn.preprocessing import MinMaxScaler, RobustScaler, MaxAbsScaler
import pandas as pd
logging.basicConfig(filename='data_preprocessing.log', level=logging.INFO, format='%(asctime)s - %(message)s')
data = {'feature1': [10, 20, 30, 40, 50],
        'feature2': [1, 2, 3, 4, 5]}
df = pd.DataFrame(data)
def log_preprocessing_step(step_name, details):
    logging.info(f"Step: {step_name}, Details: {details}")
scaler_minmax = MinMaxScaler()
scaler_robust = RobustScaler()
scaler_maxabs = MaxAbsScaler()
scaler_minmax.fit(df)
scaled_data_minmax = scaler_minmax.transform(df)
log_preprocessing_step("Min-Max Scaling", {"min": scaler_minmax.data_min_, "max": scaler_minmax.data_max_})
scaler_robust.fit(df)
scaled_data_robust = scaler_robust.transform(df)
log_preprocessing_step("Robust Scaling", {"median": scaler_robust.center_, "IQR": scaler_robust.scale_})
scaler_maxabs.fit(df)
scaled_data_maxabs = scaler_maxabs.transform(df)
log_preprocessing_step("MaxAbs Scaling", {"max_abs": scaler_maxabs.max_abs_})
print(scaled_data_minmax)
print(scaled_data_robust)
print(scaled_data_maxabs)

#.
import pickle
def store_transformation_parameters(scaler, filename):
    with open(filename, 'wb') as file:
        pickle.dump(scaler, file)
store_transformation_parameters(scaler_minmax, 'minmax_scaler.pkl')
store_transformation_parameters(scaler_robust, 'robust_scaler.pkl')
store_transformation_parameters(scaler_maxabs, 'maxabs_scaler.pkl')
def load_transformation_parameters(filename):
    with open(filename, 'rb') as file:
        scaler = pickle.load(file)
    return scaler
loaded_scaler_minmax = load_transformation_parameters('minmax_scaler.pkl')
print(loaded_scaler_minmax.data_min_, loaded_scaler_minmax.data_max_)